##4.1 A simple height map


##The <code><span style="background-color:lightgrey">lidar_camera.dat</span></code> file

The contents of the <code><span style="background-color:lightgrey">lidar_camera.dat</span></code> are this:

<code>
